In [1]:
from collections import deque, namedtuple
from IPython.display import HTML, display
from IPython.core.debugger import set_trace
from tqdm import tqdm

import pandas as pd
import numpy as np
import requests
import sqlite3
import urllib

In [2]:
SAMPLE_ID = 25224070
START = 20140101
END = 20200301

DB = './db.sqlite'
TABLES = ['PhilosophicalSchool', 'Philosopher', 'NotableIdea', 'MainInterest', 'Era', 'Work']
TABLE_NAME = 'WikiPagePopularity'
TEMP_FETCH = './_temp_fetch.csv'

In [3]:
def fetch_title(page_id):
    r = requests.get('https://en.wikipedia.org/w/api.php', params={
        'action': 'query',
        'prop': 'info',
        'pageids': page_id,
        'format': 'json'
    })
    try:
         return r.json()['query']['pages'][str(page_id)]['title']
    except Exception as exp:
        print('Error:', page_id)
        display(page_id)
    
title = fetch_title(SAMPLE_ID)
print(title)

Wikipedia:Pageview statistics


In [4]:
def fetch_popularity(page_title):
    page_title = urllib.parse.quote_plus(page_title.replace(' ', '_'))
    r = requests.get(
        'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/'
        f'en.wikipedia.org/all-access/all-agents/{page_title}/monthly/{START}/{END}'
    )
    try:
        res = r.json()['items']
    except Exception as exp:
        print('Error:', page_title)
        display(exp)
        return None
    df = pd.DataFrame(res)
    return df.views.sum()

fetch_popularity(title)

166605

In [5]:
def get_ids():
    try:
        conn = sqlite3.connect(DB)
        ids = set()
        for table_name in TABLES:
            df = pd.read_sql(f'SELECT * FROM {table_name}', conn)
            df['wikiPageID'].dropna().apply(lambda val: ids.add(int(val)))
    finally:
        conn.close
    ids = list(ids)
    ids.sort()
    return ids
        
ids = get_ids()

In [6]:
fetched = deque()
Popularity = namedtuple('Popularity', ['wikiPageID', 'total_visits'])

In [7]:
def restore_fetched(ids, fetched):
    df = pd.read_csv(TEMP_FETCH)
    print('Total:', len(ids))
    nones = 0
    for id_, pop in df.itertuples(index = False):
        if pop is not None and pop > 0:
            fetched.append(Popularity(id_, pop))
            ids.remove(id_)
        else:
            nones += 1
    print('Restored:', len(fetched))
    print('None:', nones)
    print('Remaining:', len(ids))
        
restore_fetched(ids, fetched)

Total: 5102
Restored: 5062
None: 40
Remaining: 40


In [8]:
def fetch_ids_popularity(ids, fetched):
    for id_ in tqdm(ids):
        title = fetch_title(id_)
        if title is None:
            fetched.append(Popularity(id_, None))
            continue
        popularity = fetch_popularity(title)
        if popularity is None:
            fetched.append(Popularity(id_, None))
            continue
        fetched.append(Popularity(id_, popularity))
    return fetched

fetched = fetch_ids_popularity(ids, fetched)

  2%|▎         | 1/40 [00:01<00:58,  1.51s/it]

Error: 286153


286153

  5%|▌         | 2/40 [00:02<00:46,  1.22s/it]

Error: 1486408


...

51368906

 98%|█████████▊| 39/40 [00:33<00:01,  1.28s/it]

Error: 51443180


51443180

100%|██████████| 40/40 [00:33<00:00,  1.19it/s]


In [9]:
def save_fetched(fetched):
    df = pd.DataFrame(fetched)
    df.to_csv(TEMP_FETCH, index=False)
    
save_fetched(fetched)

In [11]:
def add_fetched_to_db(fetched):
    df = pd.DataFrame(fetched)
    df = df.set_index('wikiPageID')
    try:
        conn = sqlite3.connect(DB)
        df.to_sql(TABLE_NAME, conn, if_exists='replace')
        conn.commit()
    finally:
        conn.close()
        
add_fetched_to_db(fetched)